<a href="https://colab.research.google.com/github/vypham2401/data-analytics/blob/main/SENKENII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

API Work 

In [ ]:
import requests
import pandas as pd
import time
import json
from time import sleep
from pandas import json_normalize 

In [ ]:
#function for reassigning keys in json response
def relabel(dictionary, token_id, pool_id, resolution):
    full_labels =["Datetime", "Open", "High", "Low", "Close", "Volume", "Time of Request", "Token_ID", "Pool_ID", "Resolution"]
    #decided to add this column because the datetime the json returns is the timestamp for the ohlc. Sometimes it wont change
    #and so it will give us the same time stamp. Adding this column gives us the over time aspect.
    dictionary.update({"Time of Request" : pd.to_datetime((int(time.time())//60 * 60), unit='s')})
    dictionary.update({"Token_ID" : token_id})
    dictionary.update({"Pool_ID" : pool_id})
    dictionary.update({"Resolution" : resolution})
    return dict(zip(full_labels, list(dictionary.values())))

In [ ]:
#initialize query parameters

#token id for NCT 
token_id="2609763"
#pool id, don't 100% know what this is 
pool_id="148205730"
#frequency, here it is 1 minute
resolution="1"
#the time stamp of the last change in ohlc
from_time="1660403580"
#the time stamp of the minute we want to check to
to_time="1660529400"

#request url
url = "https://app.geckoterminal.com/contracts/1/candlesticks.json?token_id=2609763&pool_id=148205730&resolution=1&from=1660403580&to=1660527600&for_update=true"
#initialize request headers
headers = {
	"Request Headers (542 B)": {
		"headers": [
			{
				"name": "Accept",
				"value": "*/*"
			},
			{
				"name": "Accept-Encoding",
				"value": "gzip, deflate, br"
			},
			{
				"name": "Accept-Language",
				"value": "en-US,en;q=0.5"
			},
			{
				"name": "Connection",
				"value": "keep-alive"
			},
			{
				"name": "DNT",
				"value": "1"
			},
			{
				"name": "Host",
				"value": "app.geckoterminal.com"
			},
			{
				"name": "Origin",
				"value": "https://www.geckoterminal.com"
			},
			{
				"name": "Referer",
				"value": "https://www.geckoterminal.com/"
			},
			{
				"name": "Sec-Fetch-Dest",
				"value": "empty"
			},
			{
				"name": "Sec-Fetch-Mode",
				"value": "cors"
			},
			{
				"name": "Sec-Fetch-Site",
				"value": "same-site"
			},
			{
				"name": "Sec-GPC",
				"value": "1"
			},
			{
				"name": "TE",
				"value": "trailers"
			},
			{
				"name": "User-Agent",
				"value": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:103.0) Gecko/20100101 Firefox/103.0"
			}
		]
	}
}

#make our first request and get the data frame started
r = requests.get(url,headers)
#read json down to the data we need
pre_relabel_info = json.loads(r.text)["data"][0]
print(json.loads(r.text)["meta"])
trys = {}
if (trys == json.loads(r.text)["meta"]):
    print("don't need quotes")
#make it pretty
info = relabel(pre_relabel_info, token_id, pool_id, resolution)
#create the dataframe
df = json_normalize(info)

df.head()

,Datetime,Open,High,Low,Close,Volume,Time of Request,Token_ID,Pool_ID,Resolution
0,2022-08-12T19:51:00.000Z,2.307072,2.307072,2.307072,2.307072,184.071002,2022-08-15 13:29:00,2609763,148205730,1


In [ ]:
#initiialize a previous response variable that we can compare to     
previous_response_info = "None"
conn = sqlite3.connect('NCT.db')

while True:
    
    #build url using variables instead of hard typing it
    url ="https://app.geckoterminal.com/contracts/1/candlesticks.json?token_id="+token_id+"&pool_id="+pool_id+"&resolution="+resolution+"&from="+from_time+"&to="+to_time+"&for_update=true"

    #make new request    
    current_response = requests.get(url, headers)
    pre_relabel_current_response_info = json.loads(current_response.text)["data"][0]
    #read json down to the data we need
#     if (json.loads(current_response.text)["meta"] != {}):
#         good_response = False
#         print("got a bad response for " + to_time)
#         print(url)
#         while (good_response != True): 
#             time.sleep(60)
#             to_time = str(int(to_time)+60)
#             current_response = requests.get(url,headers)
#             print(to_time)
#             if (json.loads(current_response.text)["meta"] == {}):
#                 good_response = True
#             else:
#                 print("it messed up again")
        
#     print(current_response.text)
    current_response_info = json.loads(current_response.text)["data"][0]
    #if we get new ohlc values then the from time will need to be adjsuted to the time we checked to and become the new most 
    #timestamp of ohlc change
    if ((current_response_info != pre_relabel_current_response_info) & (pre_relabel_current_response_info != "None")): 
        print(previous_response_info)
        print(current_response_info)
        print("we changed the from time")
        from_time = to_time
    #make it match the format of the data frame
    current_response_info = relabel(current_response_info, token_id, pool_id, resolution)
    #create a new dataframe line
    df2 = json_normalize(current_response_info)
    #append to the 1 minute tabel in the database
    df7.to_sql(name='NCT_1_Minute', con = conn, if_exists='append', index=False)
    #update the previous response
    previous_response_info = pre_relabel_current_response_info
    #add 60 seconds to the time we need to check to    
    to_time = str(int(to_time)+60)
    #chill out for a minute 
    time.sleep(60)
    df.head()



{'Datetime': '2022-08-13T15:13:00.000Z', 'Open': 2.30245508922384, 'High': 2.30245508922384, 'Low': 2.30245508922384, 'Close': 2.30245508922384, 'Volume': 216.471919098715, 'Time of Request': Timestamp('2022-08-15 13:30:00'), 'Token_ID': '2609763', 'Pool_ID': '148205730', 'Resolution': '1'}
{'Datetime': '2022-08-13T15:13:00.000Z', 'Open': 2.30245508922384, 'High': 2.30245508922384, 'Low': 2.30245508922384, 'Close': 2.30245508922384, 'Volume': 216.471919098715, 'Time of Request': Timestamp('2022-08-15 13:31:00'), 'Token_ID': '2609763', 'Pool_ID': '148205730', 'Resolution': '1'}


IndexError: ignored

In [ ]:
df.head(15)

,Datetime,Open,High,Low,Close,Volume,Time of Request,Token_ID,Pool_ID,Resolution
0,2022-08-12T19:51:00.000Z,2.307072,2.307072,2.307072,2.307072,184.071002,2022-08-15 00:42:00,2609763,148205730,1
1,2022-08-13T15:13:00.000Z,2.302455,2.302455,2.302455,2.302455,216.471919,2022-08-15 00:42:00,2609763,148205730,1
2,2022-08-13T15:13:00.000Z,2.302455,2.302455,2.302455,2.302455,216.471919,2022-08-15 00:43:00,2609763,148205730,1


In [ ]:
df.to_json(orient='index')

# API work


In [ ]:
import requests
import pandas as pd
import json as j

In [ ]:
import requests

url = "https://coingecko.p.rapidapi.com/coins/toucan-protocol-nature-carbon-tonne/tickers"

headers = {
	"X-RapidAPI-Key": "d29c6132c9mshed07801a213d72cp165fedjsne274d0c24d5b",
	"X-RapidAPI-Host": "coingecko.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)

print(response.text)

{"name":"Toucan Protocol: Nature Carbon Tonne","tickers":[{"base":"0X2791BCA1F2DE4661ED88A30C99A7A9449AA84174","target":"0XD838290E877E0188A4A44700463419ED96C16107","market":{"name":"Sushiswap (Polygon POS)","identifier":"sushiswap_polygon_pos","has_trading_incentive":false},"last":0.46144628490397,"volume":19172.169067,"converted_last":{"btc":9.575e-05,"eth":0.00135965,"usd":2.17},"converted_volume":{"btc":0.84737778,"eth":12.033308,"usd":19185.08},"trust_score":"green","bid_ask_spread_percentage":0.603095,"timestamp":"2022-08-04T17:11:36+00:00","last_traded_at":"2022-08-04T17:11:36+00:00","last_fetch_at":"2022-08-04T17:27:28+00:00","is_anomaly":false,"is_stale":false,"trade_url":"https://app.sushi.com/en/swap?inputCurrency=0x2791bca1f2de4661ed88a30c99a7a9449aa84174\u0026outputCurrency=0xd838290e877e0188a4a44700463419ed96c16107","token_info_url":null,"coin_id":"usd-coin","target_coin_id":"toucan-protocol-nature-carbon-tonne"}]}


In [ ]:
pip install pycoingecko


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pycoingecko import CoinGeckoAPI
import time as t
import datetime as dt
import plotly.graph_objects as go
from plotly.offline import plot
import pandas as pd

In [ ]:
data =CoinGeckoAPI()


In [ ]:
data.ping()

{'gecko_says': '(V3) To the Moon!'}

In [ ]:
#get the coin
NCT = data.get_coin_by_id(id = 'toucan-protocol-nature-carbon-tonne')
print(NCT)

{'id': 'toucan-protocol-nature-carbon-tonne', 'symbol': 'nct', 'name': 'Toucan Protocol: Nature Carbon Tonne', 'asset_platform_id': 'polygon-pos', 'platforms': {'polygon-pos': '0xd838290e877e0188a4a44700463419ed96c16107'}, 'block_time_in_minutes': 0, 'hashing_algorithm': None, 'categories': ['Polygon Ecosystem'], 'public_notice': None, 'additional_notices': ["Kindly be aware of <a href='https://www.coingecko.com/en/glossary/rug-pulled' target='_blank'>liquidity-related risks</a>. This notice is not directed at any project in particular, and is more of a cautionary reminder."], 'localization': {'en': 'Toucan Protocol: Nature Carbon Tonne', 'de': 'Toucan Protocol: Nature Carbon Tonne', 'es': 'Toucan Protocol: Nature Carbon Tonne', 'fr': 'Toucan Protocol: Nature Carbon Tonne', 'it': 'Toucan Protocol: Nature Carbon Tonne', 'pl': 'Toucan Protocol: Nature Carbon Tonne', 'ro': 'Toucan Protocol: Nature Carbon Tonne', 'hu': 'Toucan Protocol: Nature Carbon Tonne', 'nl': 'Toucan Protocol: Nature 

In [ ]:
#get NCT price and market chart
NCTmarket = data.get_coin_market_chart_by_id(id = 'toucan-protocol-nature-carbon-tonne', vs_currency = 'usd', days = 10)
print(NCTmarket)



{'prices': [[1659366374757, 2.412546068777908], [1659369978695, 2.42675085005286], [1659373564643, 2.406179899163057], [1659377166873, 2.4173351519646133], [1659380789002, 2.4156340832339893], [1659384394413, 2.4179258929297633], [1659387988237, 2.4270613259459304], [1659391550391, 2.41507119436831], [1659395167675, 2.4202255464078757], [1659398755145, 2.418456461074281], [1659402375873, 2.429970710426522], [1659409589333, 2.407424323642165], [1659413339267, 2.426444446506117], [1659416746424, 2.4211330627545626], [1659420347252, 2.418821121491276], [1659423969293, 2.42187016855457], [1659427579976, 2.4201265181113034], [1659431178231, 2.411911746143247], [1659434775932, 2.414425860045286], [1659438345988, 2.4254705684240543], [1659442027059, 2.441398572760218], [1659445554785, 2.4359823244302063], [1659449141274, 2.415866145572675], [1659452766838, 2.4567434609335193], [1659456369640, 2.435247000760929], [1659460010755, 2.4295087711989343], [1659463602883, 2.41542435518496], [16594671

In [ ]:
#create DataFrame for NCT market chart
# but dataframe only record price change every 1 hr
#need to separate timestamp into a separate column/ and change it to normal timestamp
coinDataFrame = pd.DataFrame.from_dict(NCTmarket)
coinDataFrame

,prices,market_caps,total_volumes
0,"[1659366374757, 2.412546068777908]","[1659366374757, 0.0]","[1659366374757, 125.04558066279412]"
1,"[1659369978695, 2.42675085005286]","[1659369978695, 0.0]","[1659369978695, 125.78183401177749]"
2,"[1659373564643, 2.406179899163057]","[1659373564643, 0.0]","[1659373564643, 175.93369870683097]"
3,"[1659377166873, 2.4173351519646133]","[1659377166873, 0.0]","[1659377166873, 176.74934216144968]"
4,"[1659380789002, 2.4156340832339893]","[1659380789002, 0.0]","[1659380789002, 176.62496438170115]"
...,...,...,...
233,"[1660215969648, 2.3076743551793606]","[1660215969648, 0.0]","[1660215969648, 253.6825348790897]"
234,"[1660219576563, 2.3196827964166293]","[1660219576563, 0.0]","[1660219576563, 255.00262226758096]"
235,"[1660223170690, 2.3158515928590737]","[1660223170690, 0.0]","[1660223170690, 257.23178516689586]"
236,"[1660226780983, 2.3071984610663403]","[1660226780983, 0.0]","[1660226780983, 264.19196225952373]"


In [ ]:
PriceRequest = data.get_price(ids='toucan-protocol-nature-carbon-tonne', vs_currencies = 'usd', include_market_cap = True, include_24hr_vol = True, include_24hr_change = True, include_last_updated_at = True)
print(PriceRequest)

{'toucan-protocol-nature-carbon-tonne': {'usd': 2.32, 'usd_market_cap': 0.0, 'usd_24h_vol': 265.1553223541727, 'usd_24h_change': 0.5120585175151109, 'last_updated_at': 1660228850}}


# Store price/Timestamp for 1m:

In [ ]:
#need to use pickles (pickle is a process of converting a Python object into a byte stream to store it in a file/database) 
#create a database(to store timestamps, and NCT price movements)
#create an automatic system to query API and update new data

In [ ]:
#import all the important platforms
import pickle
import numpy as np
import sqlite3
import pandas as pd
import sqlalchemy


In [ ]:
df.head()

,Datetime,Open,High,Low,Close,Volume,Time of Request,Token_ID,Pool_ID,Resolution
0,2022-08-12T19:51:00.000Z,2.307072,2.307072,2.307072,2.307072,184.071002,2022-08-15 13:29:00,2609763,148205730,1
1,2022-08-13T15:13:00.000Z,2.302455,2.302455,2.302455,2.302455,216.471919,2022-08-15 13:30:00,2609763,148205730,1
2,2022-08-13T15:13:00.000Z,2.302455,2.302455,2.302455,2.302455,216.471919,2022-08-15 13:31:00,2609763,148205730,1


In [ ]:
#database part
conn = sqlite3.connect('NCT.db')
cur =conn.cursor()
print("Opened database successfully;")


Opened database successfully;


In [ ]:
conn.execute('''CREATE TABLE Chart
         (DATETIME       INT     NOT NULL,
         OPEN           FLOAT    NOT NULL,
         HIGH          FLOAT     NOT NULL,
         LOW         FLOAT,
         CLOSE       FLOAT,
         VOLUME      FLOAT,
         TIME OF REQUEST      TEXT,
         TOKEN_ID          INT,
         POOL_ID           INT,
         RESOLUTION        INT);''')
print ("Table created successfully;")
conn.close()


ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 32))



OperationalError: ignored

Another way


In [ ]:
for row in df.itertuples():
    insert_sql =f"INSERT INTO Chart (DATETIME, OPEN, HIGH, LOW, CLOSE, VOLUME, TIME OF REQUEST, TOKEN_ID, POOL_ID, RESOLUTION) VALUES ({row[1]}, {row[2]}, {row[3]}, {row[4]}, {row[5]}, {row[6]}, {row[7]}, {row[8]}, {row[9]}, {row[10]})"
    conn.execute(insert_sql)

In [ ]:
df.to_sql(name="NCT price chart", con=conn, if_exists='replace', index=False)

In [ ]:
conn.commit()

# Historical Database for 1W, 1M, 1Y
